In [1]:
import os, sys
import time, math
import argparse, random
from math import exp
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.backends import cudnn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as tfs
from torchvision.transforms import ToPILImage
from torchvision.transforms import functional as FF
import torchvision.utils as vutils
from torchvision.utils import make_grid
from torchvision.models import vgg16

from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Settings ⚙️

In [2]:
# Device name
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Num residual_groups
gps = 3
# Num residual_blocks
blocks = 19
# Directory of test imgs
img_dir = '../input/synthetic-objective-testing-set-sots-reside/outdoor/hazy/'
# Pre-trained checkpoint dir
pretrained_model_dir = '../input/ffanet-pretrained-weights/' + f'ots_train_ffa_{gps}_{blocks}.pk'
# Output dir to save predicted de-hazed images
output_dir = f'pred_FFA_ots/'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

### Utility Functions

In [3]:
def tensorShow(tensors,titles=None):
    '''t:BCWH'''
    fig=plt.figure()
    for tensor, title, i in zip(tensors, titles, range(len(tensors))):
        img = make_grid(tensor)
        npimg = img.numpy()
        ax = fig.add_subplot(211+i)
        ax.imshow(np.transpose(npimg, (1, 2, 0)))
        ax.set_title(title)
    plt.show()

### Model Definition

In [4]:
def default_conv(in_channels, out_channels, kernel_size, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size, padding=(kernel_size//2), bias=bias)
    
    
class PALayer(nn.Module):
    def __init__(self, channel):
        super(PALayer, self).__init__()
        self.pa = nn.Sequential(
                nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 8, 1, 1, padding=0, bias=True),
                nn.Sigmoid()
        )
    def forward(self, x):
        y = self.pa(x)
        return x * y

    
class CALayer(nn.Module):
    def __init__(self, channel):
        super(CALayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.ca = nn.Sequential(
                nn.Conv2d(channel, channel // 8, 1, padding=0, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 8, channel, 1, padding=0, bias=True),
                nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.ca(y)
        return x * y

    
class Block(nn.Module):
    def __init__(self, conv, dim, kernel_size,):
        super(Block, self).__init__()
        self.conv1 = conv(dim, dim, kernel_size, bias=True)
        self.act1 = nn.ReLU(inplace=True)
        self.conv2 = conv(dim, dim, kernel_size, bias=True)
        self.calayer = CALayer(dim)
        self.palayer = PALayer(dim)

    def forward(self, x):
        res = self.act1(self.conv1(x))
        res = res+x 
        res = self.conv2(res)
        res = self.calayer(res)
        res = self.palayer(res)
        res += x 
        return res

    
class Group(nn.Module):
    def __init__(self, conv, dim, kernel_size, blocks):
        super(Group, self).__init__()
        modules = [Block(conv, dim, kernel_size)  for _ in range(blocks)]
        modules.append(conv(dim, dim, kernel_size))
        self.gp = nn.Sequential(*modules)

    def forward(self, x):
        res = self.gp(x)
        res += x
        return res

    
class FFA(nn.Module):
    def __init__(self,gps,blocks,conv=default_conv):
        super(FFA, self).__init__()
        self.gps = gps
        self.dim = 64
        kernel_size = 3
        pre_process = [conv(3, self.dim, kernel_size)]
        assert self.gps==3
        self.g1 = Group(conv, self.dim, kernel_size,blocks=blocks)
        self.g2 = Group(conv, self.dim, kernel_size,blocks=blocks)
        self.g3 = Group(conv, self.dim, kernel_size,blocks=blocks)
        self.ca = nn.Sequential(*[
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(self.dim*self.gps,self.dim//16,1,padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.dim//16, self.dim*self.gps, 1, padding=0, bias=True),
            nn.Sigmoid()
            ])
        self.palayer = PALayer(self.dim)

        post_process = [
            conv(self.dim, self.dim, kernel_size),
            conv(self.dim, 3, kernel_size)]

        self.pre = nn.Sequential(*pre_process)
        self.post = nn.Sequential(*post_process)

    def forward(self, x1):
        x = self.pre(x1)
        res1 = self.g1(x)
        res2 = self.g2(res1)
        res3 = self.g3(res2)
        w = self.ca(torch.cat([res1,res2,res3],dim=1))
        w = w.view(-1,self.gps, self.dim)[:,:,:,None,None]
        out = w[:,0,::] * res1 + w[:,1,::] * res2+w[:,2,::] * res3
        out = self.palayer(out)
        x = self.post(out)
        return x + x1

### Test FFA-Net

In [5]:
ckp = torch.load(pretrained_model_dir, map_location=device)
net = FFA(gps=gps, blocks=blocks)
net = nn.DataParallel(net)
net.load_state_dict(ckp['model'])
net.eval()

img_paths = sorted(os.listdir(img_dir))

for im in img_paths:
    haze = Image.open(img_dir+im)
    haze1 = tfs.Compose([
        tfs.ToTensor(),
        tfs.Normalize(mean=[0.64, 0.6, 0.58],std=[0.14,0.15, 0.152])
    ])(haze)[None,::]
    haze_no = tfs.ToTensor()(haze)[None,::]
    with torch.no_grad():
        pred = net(haze1)
    ts = torch.squeeze(pred.clamp(0,1).cpu())
    # tensorShow([haze_no, pred.clamp(0,1).cpu()],['haze', 'pred'])
    
    haze_no = make_grid(haze_no, nrow=1, normalize=True)
    ts = make_grid(ts, nrow=1, normalize=True)
    image_grid = torch.cat((haze_no, ts), -1)
    vutils.save_image(image_grid, output_dir+im.split('.')[0]+'_FFA.png')

In [6]:
!pip install scikit-image

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [8]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def calculate_psnr(img1, img2):
    psnr_value = peak_signal_noise_ratio(img1, img2)
    return psnr_value

def calculate_ssim(img1, img2):
    ssim_value, _ = structural_similarity(img1, img2, full=True, multichannel=True)
    return ssim_value

# PSNR and SSIM calculations
for im in img_paths:
    haze = Image.open(img_dir + im)
    haze1 = tfs.Compose([
        tfs.ToTensor(),
        tfs.Normalize(mean=[0.64, 0.6, 0.58], std=[0.14, 0.15, 0.152])
    ])(haze)[None, ::]
    
    with torch.no_grad():
        pred = net(haze1)
    
    haze_no = tfs.ToTensor()(haze)[None, ::]
    haze_no_np = np.transpose(make_grid(haze_no, nrow=1, normalize=True).numpy(), (1, 2, 0))
    
    pred_np = np.transpose(make_grid(pred.clamp(0, 1).cpu(), nrow=1, normalize=True).numpy(), (1, 2, 0))
    
    # Calculate PSNR
    psnr_value = calculate_psnr(haze_no_np, pred_np)
    
    # Calculate SSIM
    ssim_value = calculate_ssim(haze_no_np, pred_np)
    
    print(f'{im}: PSNR: {psnr_value:.4f}, SSIM: {ssim_value:.4f}')


0001_0.8_0.2.jpg: PSNR: 25.8832, SSIM: 0.9776
0002_0.8_0.08.jpg: PSNR: 35.5592, SSIM: 0.9931
0003_0.8_0.2.jpg: PSNR: 30.8051, SSIM: 0.9926
0004_0.9_0.12.jpg: PSNR: 29.3779, SSIM: 0.9883
0006_0.85_0.08.jpg: PSNR: 27.4197, SSIM: 0.9786
0007_0.9_0.16.jpg: PSNR: 17.5722, SSIM: 0.8912
0009_0.8_0.16.jpg: PSNR: 16.5148, SSIM: 0.8671
0010_0.95_0.16.jpg: PSNR: 13.8652, SSIM: 0.7475
0011_0.95_0.16.jpg: PSNR: 17.9860, SSIM: 0.8361
0014_0.8_0.12.jpg: PSNR: 25.5561, SSIM: 0.9852
0016_0.8_0.08.jpg: PSNR: 30.2669, SSIM: 0.9899
0017_0.9_0.08.jpg: PSNR: 26.4150, SSIM: 0.9365
0018_0.9_0.2.jpg: PSNR: 23.1662, SSIM: 0.9530
0019_0.8_0.16.jpg: PSNR: 20.3324, SSIM: 0.9250
0021_0.8_0.08.jpg: PSNR: 37.2400, SSIM: 0.9973
0022_0.8_0.08.jpg: PSNR: 36.7459, SSIM: 0.9971
0023_0.85_0.12.jpg: PSNR: 31.1371, SSIM: 0.9801
0024_0.85_0.08.jpg: PSNR: 31.3769, SSIM: 0.9797
0025_0.8_0.16.jpg: PSNR: 21.7652, SSIM: 0.9296
0026_0.95_0.08.jpg: PSNR: 29.3981, SSIM: 0.9819
0029_0.85_0.08.jpg: PSNR: 23.4643, SSIM: 0.9508
0030_0.95

KeyboardInterrupt: 

In [10]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from tabulate import tabulate

def calculate_psnr(img1, img2):
    psnr_value = peak_signal_noise_ratio(img1, img2)
    return psnr_value

def calculate_ssim(img1, img2):
    ssim_value, _ = structural_similarity(img1, img2, full=True, multichannel=True)
    return ssim_value

# Table headers
table_headers = ["Image", "PSNR", "SSIM"]

# Data list for the table
table_data = []

# PSNR and SSIM calculations for the first 20 images
for im in img_paths[:20]:
    haze = Image.open(img_dir + im)
    haze1 = tfs.Compose([
        tfs.ToTensor(),
        tfs.Normalize(mean=[0.64, 0.6, 0.58], std=[0.14, 0.15, 0.152])
    ])(haze)[None, ::]
    
    with torch.no_grad():
        pred = net(haze1)
    
    haze_no = tfs.ToTensor()(haze)[None, ::]
    haze_no_np = np.transpose(make_grid(haze_no, nrow=1, normalize=True).numpy(), (1, 2, 0))
    
    pred_np = np.transpose(make_grid(pred.clamp(0, 1).cpu(), nrow=1, normalize=True).numpy(), (1, 2, 0))
    
    # Calculate PSNR and SSIM
    psnr_value = calculate_psnr(haze_no_np, pred_np)
    ssim_value = calculate_ssim(haze_no_np, pred_np)
    
    # Append data to the table
    table_data.append([im, f"{psnr_value:.4f}", f"{ssim_value:.4f}"])

# Display the table
print(tabulate(table_data, headers=table_headers, tablefmt="grid"))


+--------------------+---------+--------+
| Image              |    PSNR |   SSIM |
+====================+=========+========+
| 0001_0.8_0.2.jpg   | 25.8832 | 0.9776 |
+--------------------+---------+--------+
| 0002_0.8_0.08.jpg  | 35.5592 | 0.9931 |
+--------------------+---------+--------+
| 0003_0.8_0.2.jpg   | 30.8051 | 0.9926 |
+--------------------+---------+--------+
| 0004_0.9_0.12.jpg  | 29.3779 | 0.9883 |
+--------------------+---------+--------+
| 0006_0.85_0.08.jpg | 27.4197 | 0.9786 |
+--------------------+---------+--------+
| 0007_0.9_0.16.jpg  | 17.5722 | 0.8912 |
+--------------------+---------+--------+
| 0009_0.8_0.16.jpg  | 16.5148 | 0.8671 |
+--------------------+---------+--------+
| 0010_0.95_0.16.jpg | 13.8652 | 0.7475 |
+--------------------+---------+--------+
| 0011_0.95_0.16.jpg | 17.986  | 0.8361 |
+--------------------+---------+--------+
| 0014_0.8_0.12.jpg  | 25.5561 | 0.9852 |
+--------------------+---------+--------+
| 0016_0.8_0.08.jpg  | 30.2669 | 0

In [15]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

def display_images(clear_images, hazy_images, nrow=2, title1="Clear Images", title2="Hazy Images"):
    clear_grid = make_grid([tfs.ToTensor()(img) for img in clear_images], nrow=nrow, normalize=True)
    hazy_grid = make_grid([tfs.ToTensor()(img) for img in hazy_images], nrow=nrow, normalize=True)

    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 5))
    axes[0].imshow(np.transpose(clear_grid.numpy(), (1, 2, 0)))
    axes[0].set_title(title1)
    axes[0].axis('off')

    axes[1].imshow(np.transpose(hazy_grid.numpy(), (1, 2, 0)))
    axes[1].set_title(title2)
    axes[1].axis('off')

    plt.show()

# Load clear and hazy images from the dataset
clear_images = []
hazy_images = []

for im in img_paths[:20]:  # Assuming the first 20 images
    haze = Image.open(img_dir + im)

    clear_image = tfs.ToTensor()(haze)
    hazy_image = tfs.Compose([
        tfs.ToTensor(),
        tfs.Normalize(mean=[0.64, 0.6, 0.58], std=[0.14, 0.15, 0.152])
    ])(haze)

    # Convert PyTorch tensors back to PIL Images for resizing
    clear_image = TF.to_pil_image(clear_image)
    hazy_image = TF.to_pil_image(hazy_image)

    # Resize the hazy image to match the size of the clear image
    hazy_image = TF.resize(hazy_image, clear_image.size)

    clear_images.append(clear_image)
    hazy_images.append(hazy_image)

# Display clear and hazy images in a grid
display_images(clear_images, hazy_images)


RuntimeError: stack expects each tensor to be equal size, but got [3, 413, 550] at entry 0 and [3, 309, 550] at entry 2